In [ ]:
import json 

from pymongo import MongoClient
from bson.objectid import ObjectId
#used mongo to access online storage, fill in brackets in mongoclient() to connect to your own db
conn = MongoClient()
db = conn.study  #连接mydb数据库，没有则自动创建

In [ ]:
import jieba
import synonyms
import re
# import Cilin similarity for testing.
from cilin import CilinSimilarity

# initialize stop words dictionary, this includes common meaningless words after translation from English to Chinese
except_words = ['的', '，', '地', '…','...', '或', '；', '为', '上', '(', ')', ' ', '大量'
        , '使', '发生', '在', '有', '把', '不', 'adv', '等', 'adj', '.' ,'（', 'vi','vt', 'phr','某人','']
def comparemax(word1, word2):
    cs = CilinSimilarity()
    # clean out extra spaces around word translations
    word1 = word1.strip()
    word2 = word2.strip()
    clean_word1 = ''.join(re.findall(r'[\u4e00-\u9fa5]', word1))
    # tokenize words for sub-component similarity comparisons
    final_word1 = list(jieba.lcut(clean_word1))
    #print(final_word1)
    clean_word2 = ''.join(re.findall(r'[\u4e00-\u9fa5]', word2))
    final_word2 = list(jieba.lcut(clean_word2))

    #print(final_word2)
    similarity = 0
    # compare similarities according to sub-components
    for word in final_word1:
        #skip stop words
        if word in except_words:# or len(word) == 1:
            continue
        word_sim = 0
        for word2 in final_word2:
            if word2 in except_words:# or len(word) == 1:
                continue
            if word == word2:
                word_sim = 1
            sim = cs.similarity(word, word2)
#            if sim > 0.8 and sim < 1:
#             print(word, word2, sim)
            #choose max score as the similarity score
            word_sim = max(sim, word_sim)
        similarity = max(word_sim, similarity)
    pair = [clean_word2, similarity]
    print(similarity)
    if similarity >= 0.8:
        return True
    return False

print(comparemax("不行","不好"))

In [ ]:
# This is a different approach involving weighted score when comparing similarities
import jieba
import synonyms
import re
from cilin import CilinSimilarity

except_words = ['的', '，', '地', '…','...', '或', '；', '为', '上', '(', ')', ' ', '大量'
        , '使', '发生', '在', '有', '把', '不', 'adv', '等', 'adj', '.' ,'（', 'vi','vt', 'phr','某人','']

def compare(word1, word2):
    cs = CilinSimilarity()
    word1 = word1.strip()
    word2 = word2.strip()
    clean_word1 = ''.join(re.findall(r'[\u4e00-\u9fa5]', word1))
    final_word1 = list(jieba.lcut(clean_word1))
    clean_word2 = ''.join(re.findall(r'[\u4e00-\u9fa5]', word2))
    final_word2 = list(jieba.lcut(clean_word2))

    index = 0
    for i in range(len(final_word1)):
        if (final_word1[index] in except_words):
            final_word1.remove(final_word1[index])
        else:
            index += 1
    index2 = 0
    for j in range(len(final_word2)):
        if (final_word2[index2] in except_words):
            final_word2.remove(final_word2[index2])
        else:
            index2 += 1
            
    similarity = 0

    for word in final_word1:
        word_sim = 0
        for word2 in final_word2:
            # apply double weights to sub-components with similarity score = 1
            if cs.similarity(word, word2) == 1 or word == word2:
                word_sim += 2
            word_sim += cs.similarity(word, word2)
        similarity += word_sim
    # weight the total score
    similarity /= (len(final_word1)*len(final_word2))
    pair = [clean_word2, similarity]
    if similarity >= 0.8:
        return True
    return False

In [ ]:
# an example using self-created dictionary
words = db.words.find({
})

# every word should have its chinese translations and option candidates for multiple choice questions, our task is to eliminate options that are too similar to the correct answer.
for word in words:
        #optionRows = db.word_options.find({'word_id': str(word['_id'])})
        optionWords = db.words.find({'_id': {'$in': [x['option_id'] for x in optionRows] }})
        options = []

        for optionWord in optionWords:
            for de in optionWord['definitions']:
                options.append({'chinese':de['chinese'], 'option_id': optionWord['_id'], 'word': optionWord['word']})
        # compare every correct translation and option translations
        for definition in word['definitions']:
            for option in options:
                if compare(definition['chinese'], option['chinese']):
                    print('原词: ', word['number'], word['word'], definition['chinese'],' ==== ',option['word'], option['chinese'])
    #                 db.word_options.delete_one({'word_id': str(word['_id']), 'option_id': option['option_id']})
'''        # 比较干扰项之间
        for option1 in options:
            for option2 in options:
                if compare(option1['chinese'], option2['chinese']) and option1 != option2:
                    print('干扰项：', word['number'], word['word'],option1['word'], option1['chinese'],' = [干扰项] ',option2['word'],
                          option2['chinese'],option2['option_id'])
    #                 db.word_options.delete_one({'word_id': str(word['_id']), 'option_id': option2['option_id']})
'''

In [ ]:
# check if there are still at least 8 options available after filtering. We would like to randomly select 4 of them for multiple choice questions
words = db.words.find({
    "wordbook_id": ObjectId('5eb9336b9f84b8d354223b8b'),
})
for word in words:
    option_count = db.word_options.find({'word_id': str(word['_id'])}).count()
    if option_count <8:
        print(word['number'],word['word'],option_count)